#### Retraining the algorithm require GPU and CUDA, if not available change the flag to GPU = 0 and CUDNN = 0 in the Makefile from darknet directory

Full code, dataset and related files are uploaded in github (master branch, not in main branch) as .zip format
https://github.com/jayasooryantm/Traffic-detection-using-computer-vision/tree/master
or use the below clone command to download directly

In [ ]:
!git clone https://github.com/jayasooryantm/Traffic-detection-using-computer-vision.git

# Data Collection

In [ ]:
#cloning OID tool from github
!git clone https://github.com/EscVM/OIDv4_ToolKit.git

In [ ]:
%cd OIDv4_ToolKit

In [ ]:
!pip3 install -r 'requirements.txt'

In [ ]:
#downloading 500 images from data repository
!python 'main.py' downloader --classes Car --type_csv train --multiclasses 1 --limit 500 -y

# Data Pre-Processing

In [ ]:
import pandas as pd
import os

In [ ]:
%cd ..

In [ ]:
#getting full path to create dataset
Directory_fullpath = !pwd
Directory_fullpath = Directory_fullpath[0]

In [ ]:
Directory_fullpath

In [ ]:
full_path_to_csv = Directory_fullpath + '/OIDv4_ToolKit/OID/csv_folder'
full_path_to_images = Directory_fullpath + '/OIDv4_ToolKit/OID/Dataset/train/Car'

In [ ]:
labels = ['Car']

In [ ]:
#loading class informations
classes = pd.read_csv(full_path_to_csv + '/' + 'class-descriptions-boxable.csv', usecols=[0, 1], header=None)

In [ ]:
encrypted_strings = []

In [ ]:
for v in labels:
    sub_classes = classes.loc[classes[1] == v]
    e = sub_classes.iloc[0][0]
    encrypted_strings.append(e)

In [ ]:
#loading annotations
annotations = pd.read_csv(full_path_to_csv + '/' + 'train-annotations-bbox.csv', usecols=['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax'])

In [ ]:
#getting only required annotations
sub_ann = annotations.loc[annotations['LabelName'].isin(encrypted_strings)].copy()

In [ ]:
sub_ann['classNumber'] = ''
sub_ann['center x'] = ''
sub_ann['center y'] = ''
sub_ann['width'] = ''
sub_ann['height'] = ''

In [ ]:
for i in range(len(encrypted_strings)):
    sub_ann.loc[sub_ann['LabelName'] == encrypted_strings[i], 'classNumber'] = i

In [ ]:
#calculating the yolo required data format
sub_ann['center x'] = (sub_ann['XMax'] + sub_ann['XMin']) / 2
sub_ann['center y'] = (sub_ann['YMax'] + sub_ann['YMin']) / 2
sub_ann['width'] = sub_ann['XMax'] - sub_ann['XMin']
sub_ann['height'] = sub_ann['YMax'] - sub_ann['YMin']

In [ ]:
r = sub_ann.loc[:, ['ImageID', 'classNumber', 'center x', 'center y', 'width', 'height']].copy()

In [ ]:
os.chdir(full_path_to_images)

In [ ]:
#generating annotation file for each image
for current_dir, dirs, files in os.walk('.'):
    for f in files:
        if f.endswith('.jpg'):
            image_name = f[:-4]
            sub_r = r.loc[r['ImageID'] == image_name]
            
            resulted_frame = sub_r.loc[:, ['classNumber',
                                           'center x',
                                           'center y',
                                           'width',
                                           'height']].copy()

            path_to_save = full_path_to_images + '/' + image_name + '.txt'

            resulted_frame.to_csv(path_to_save, header=False, index=False, sep=' ')

In [ ]:
path_collection = []

In [ ]:
#collecting all image paths
for current_dir, dirs, files in os.walk('.'):
    for f in files:
        if f.endswith('.jpg'):
            path_to_save_into_txt_files = full_path_to_images + '/' + f
            path_collection.append(path_to_save_into_txt_files + '\n')

In [ ]:
#train test split
test_path = path_collection[:int(len(path_collection) * 0.15)]
path_collection = path_collection[int(len(path_collection) * 0.15):]

In [ ]:
with open('train.txt', 'w') as train_txt:
    for e in path_collection:
        train_txt.write(e)

In [ ]:
with open('test.txt', 'w') as test_txt:
    # Going through all elements of the list
    for e in test_path:
        # Writing current path at the end of the file
        test_txt.write(e)

In [ ]:
c = 0

In [ ]:
#generting class name file
with open(full_path_to_images + '/' + 'classes.names', 'w') as names, \
     open(full_path_to_images + '/' + 'classes.txt', 'r') as txt:
    for line in txt:
        names.write(line)
        c += 1

In [ ]:
#generating cofiguration file and train test file
with open(full_path_to_images + '/' + 'custom_data.data', 'w') as data:
    data.write('classes = ' + str(c) + '\n')

    data.write('train = ' + full_path_to_images + '/' + 'train.txt' + '\n')

    data.write('valid = ' + full_path_to_images + '/' + 'test.txt' + '\n')

    data.write('names = ' + full_path_to_images + '/' + 'classes.names' + '\n')

    data.write('backup = backup')

# Transfer Learning

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
darknet_path = Directory_fullpath + '/darknet'
%cd {darknet_path}

In [ ]:
!pwd

In [ ]:
#installing darknet framework to retrain YOLOv3
!make

In [ ]:
#testing the isntallation 
!./darknet

In [ ]:
#retraining the YOLov3
!./darknet detector train cfg/custom_data.data cfg/yolov3_custom_train.cfg weights/darknet53.conv.74.weights -dont_show

In [ ]:
#getting mAP (mean Average Precision)
!./darknet detector map cfg/custom_data.data cfg/yolov3_custom_train.cfg backup/yolov3_custom_train_2000.weights